# Satellite Imagery query, parse and render
The following notebook does a few things, it explores, how to query for images with the Google Earth Engine (GEE) Python API, filter using date ranges and metadata and adding image tiles to render from GEE.

In [1]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipyleaflet
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
ee.Initialize()

In [2]:
# Function to get sizes in Human readable format
suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
def humansize(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

*While single images are great to do quick analytics, the true power of the Earth Engine environment comes with the possibility of looking at really large and heavy image collections. In GEE environment image collections have their own characteristic setup and are often composed of multiple single images. They often have the similar band structure and generally share a similar metadata structure for filtering and querying.*

In this step we query an image collection and get the size and numbe of items in a collection. 

In [3]:
#Public Image Collections
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-08-01','2018-12-01').filterMetadata('CLOUD_COVER','less_than',2)

# Private Image Collection
ps4bsr=ee.ImageCollection('projects/sat-io/open-ca/ps4bsr').filterDate('2018-08-01','2018-12-01')

# Get collection size
print('Total number of LS-8 assets with filters: '+str(l8sr.size().getInfo()))
print('Total number of PS assets with filters: '+str(ps4bsr.size().getInfo()))
print('\n'+'Total size of LS-8 collection : '+str(humansize(l8sr.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))
print('Total size of PS collection : '+str(humansize(ps4bsr.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))

Total number of LS-8 assets with filters: 10533
Total number of PS assets with filters: 767

Total size of LS-8 collection : 5.6 TB
Total size of PS collection : 119.81 GB


In [4]:
# Get sample image from collection
sample_image = ee.Image(ps4bsr.first())
pp.pprint(sample_image.getInfo())
band_names_original = sample_image.bandNames()
band_names_original.getInfo()

{'bands': [{'crs': 'EPSG:32610',
            'crs_transform': [3.0, 0.0, 597810.0, 0.0, -3.0, 4145118.0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8918, 4383],
            'id': 'b1'},
           {'crs': 'EPSG:32610',
            'crs_transform': [3.0, 0.0, 597810.0, 0.0, -3.0, 4145118.0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8918, 4383],
            'id': 'b2'},
           {'crs': 'EPSG:32610',
            'crs_transform': [3.0, 0.0, 597810.0, 0.0, -3.0, 4145118.0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [8918, 4383],
  

['b1', 'b2', 'b3', 'b4']

In [5]:
# Function to get tilelayer url from earthengine server
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [6]:
#Create a slider widget to add both Landsat 8 and PlanetScope 4B SR imagery
map1 = ipyleaflet.Map(
    center=(37.4112,-122.0634), zoom=12,
    layout={'height':'500px'},
)
ps4bsr_tile_url=GetTileLayerUrl(ps4bsr.median().visualize(min=600, max=4000, bands=['b4', 'b3', 'b2']))
l8sr_tile_url = GetTileLayerUrl(l8sr.median().visualize(min=100, max=3500, gamma=1.5, bands= ['B5', 'B3', 'B2']))  #Landsat 8 SR
left = ipyleaflet.TileLayer(url=ps4bsr_tile_url)
right=ipyleaflet.TileLayer(url=l8sr_tile_url)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map1.add_control(control)
map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# Calculate a Spectral Index (NDVI)

The normalized difference vegetation index ($NDVI$) is a band ratio that is related to the amount of green vegetation. This formula is for the four band PlanetScope Surface Reflectance imagery.

\begin{equation*}
NDVI = \frac{NIR — RED}{NIR + RED} = \frac{Band4 — Band3}{Band4 + Band3}
\end{equation*}

where $NIR$ is the near infrared band and $RED$ is red band.

In [7]:
#Generate NDVI for PlanetScope SR
img = ps4bsr.median()
red = img.select(['b3']).rename('red')
nir = img.select(['b4']).rename('nir')
ndvi = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')

In [8]:
#Create a slider widget to add both Landsat 8 and PlanetScope 4B SR imagery
map2 = ipyleaflet.Map(
    center=(37.4112,-122.0634), zoom=12,
    layout={'height':'500px'},
)
ps4bsr_tile_url=GetTileLayerUrl(ps4bsr.median().visualize(min=600, max=4000, bands=['b4', 'b3', 'b2']))
ndvi_tile_url = GetTileLayerUrl(ndvi.visualize(bands=['ndvi'], min=-0.2, max=0.6, palette='grey,yellow,green'))  #Landsat 8 SR
left = ipyleaflet.TileLayer(url=ps4bsr_tile_url)
right=ipyleaflet.TileLayer(url=ndvi_tile_url)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map2.add_control(control)
map2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Summary

We learned how to 
* Query an imagecollection using the Earth Engine API
* Get the count and size of a collection
* Adding a public dataset such as Landsat and Planet dataset under open california
* Generate a Normalized Difference Vegetation Index (NDVI) for PlanetScope 4 Band Surface Reflectance image